In [1]:
%load_ext autoreload
%autoreload 2

# 1. 修改model的load, 是base+lora的形式, 仅train lora 1
# 2. dataset不仅返回prompt_ids, 还有pixe values. 2
# 3. forward算logit时, action id, 也得转为embeddings. 3
# 4. 确定轨迹的stream length 4 
# 5. 先拿一个batch的winner, 再batch前向传播, 得到loser轨迹 

In [2]:
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoConfig, AutoImageProcessor, AutoModelForVision2Seq, AutoProcessor
from peft import PeftModel, PeftConfig
from src.data_process import TrajectoryDataset
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
import sys
sys.path.append("../..")
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union
import draccus
import numpy as np
import tqdm
from experiments.robot.robot_utils import get_model
from libero.libero import benchmark
import wandb
from experiments.robot.libero.libero_utils import (
    get_libero_dummy_action,
    get_libero_env,
    get_libero_image,
    quat2axisangle,
    save_rollout_video_CoA,
)
from experiments.robot.openvla_utils import get_processor, get_input
from experiments.robot.robot_utils import (
    DATE_TIME,
    get_action,
    get_CoA,
    get_image_resize_size,
    get_model,
    get_vla_via_lora,
    invert_gripper_action,
    normalize_gripper_action,
    set_seed_everywhere,
)

from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union
import torch
from torch.utils.data import Dataset, IterableDataset
from transformers import PreTrainedTokenizerBase, BaseImageProcessor, FeatureExtractionMixin, ProcessorMixin
from trl import DPOConfig, DPOTrainer
from typing import Union

/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-10 22:51:48.799540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-10 22:51:48.799585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-10 22:51:48.800188: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-10 22:51:48.804559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

08/10 [22:52:01] WARNING  | >> No private macro file found!                                            ]8;id=324622;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py\macros.py]8;;\:]8;id=7086;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py#53\53]8;;\

[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)


                 WARNING  | >> It is recommended to use a private macro file                           ]8;id=953317;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py\macros.py]8;;\:]8;id=365630;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py#54\54]8;;\

[robosuite WARNING] To setup, run: python /home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)


                 WARNING  | >> To setup, run: python                                                   ]8;id=702362;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py\macros.py]8;;\:]8;id=272852;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosuite/macros.py#55\55]8;;\
                          /home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/robosu             
                          ite/scripts/setup_macros.py                                                              

                 INFO     | >> No OpenGL_accelerate module loaded: No module named          ]8;id=777883;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/OpenGL/acceleratesupport.py\acceleratesupport.py]8;;\:]8;id=149229;file:///home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/OpenGL/acceleratesupport.py#24\24]8;;\
                          'OpenGL_accelerate'                                                                      

In [32]:
@dataclass
class GenerateConfig:

    # fmt: off
    vla_path: str = "openvla/openvla-7b" 
    root_dir: str = "/hdd/zijianwang"

    #################################################################################################################
    # LoRA parameters
    #################################################################################################################
    use_lora: bool = True
    lora_rank: int = 48
    lora_dropout: float = 0.0
    
    #################################################################################################################
    # Model-specific parameters
    #################################################################################################################
    model_family: str = "openvla"                    # Model family

    dataset_name: str = "libero_10_no_noops"

    pretrained_checkpoint: Union[str, Path] = os.path.join(root_dir, "openvla/FT_res/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25")     # Pretrained checkpoint path
    lora_path: str = os.path.join(root_dir, "openvla/adapter_tmp_dir/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25")
    base_vla_path: str = os.path.join(root_dir, "HF_CACHE/openvla-7b-finetuned-libero-10")

    winner_trajectory_path: str = os.path.join(root_dir, "openvla/vla-scripts/DPO/winner_trajectory")

    adapter_tmp_dir: str = os.path.join(root_dir, "openvla/adapter_tmp_dir")
    run_root_dir: str = os.path.join(root_dir, "openvla/DPO_res")

    #################################################################################################################
    load_in_8bit: bool = False                       # (For OpenVLA only) Load with 8-bit quantization
    load_in_4bit: bool = False                       # (For OpenVLA only) Load with 4-bit quantization

    center_crop: bool = True                         # Center crop? (if trained w/ random crop image aug)

    #################################################################################################################
    # Training parameters
    #################################################################################################################
    batch_size: int = 4
    grad_accumulation_steps: int = 1
    learning_rate: float = 0.0005

    #################################################################################################################
    # LIBERO environment-specific parameters
    #################################################################################################################
    task_suite_name: str = "libero_10"          # Task suite. Options: libero_spatial, libero_object, libero_goal, libero_10, libero_90
    num_steps_wait: int = 10                         # Number of steps to wait for objects to stabilize in sim
    num_trials_per_task: int = 50                    # Number of rollouts per task
    unnorm_key = task_suite_name
    #################################################################################################################
    # Utils
    #################################################################################################################
    run_id_note: Optional[str] = None                # Extra note to add in run ID for logging
    local_log_dir: str = "./experiments/logs"        # Local directory for eval logs

    use_wandb: bool = False                          # Whether to also log results in Weights & Biases
    wandb_project: str = "YOUR_WANDB_PROJECT"        # Name of W&B project to log to (use default!)
    wandb_entity: str = "YOUR_WANDB_ENTITY"          # Name of entity to log under

    seed: int = 7                                    # Random Seed (for reproducibility)

    device: str = "cuda:2"


    wandb_project = "openvla_CoA_DPO"
    wandb_entity = "15652388600"

    # fmt: on

In [4]:
def setup_model_and_config(cfg: GenerateConfig):
    """Setup and validate configuration, then load the model."""
    assert cfg.pretrained_checkpoint is not None, "cfg.pretrained_checkpoint must not be None!"
    if "image_aug" in cfg.pretrained_checkpoint:
        assert cfg.center_crop, "Expecting `center_crop==True` because model was trained with image augmentations!"
    assert not (cfg.load_in_8bit and cfg.load_in_4bit), "Cannot use both 8-bit and 4-bit quantization!"

    # Set random seed
    set_seed_everywhere(cfg.seed)

    cfg.unnorm_key = cfg.task_suite_name

    # Load model
    model = get_model(cfg)
    
    return model

def setup_logging_and_environment(cfg: GenerateConfig, model):
    """Setup logging and LIBERO environment."""
    # [OpenVLA] Check that the model contains the action un-normalization key
    if cfg.model_family == "openvla":
        # In some cases, the key must be manually modified (e.g. after training on a modified version of the dataset
        # with the suffix "_no_noops" in the dataset name)
        if cfg.unnorm_key not in model.norm_stats and f"{cfg.unnorm_key}_no_noops" in model.norm_stats:
            cfg.unnorm_key = f"{cfg.unnorm_key}_no_noops"
        assert cfg.unnorm_key in model.norm_stats, f"Action un-norm key {cfg.unnorm_key} not found in VLA `norm_stats`!"

    # [OpenVLA] Get Hugging Face processor
    processor = None
    if cfg.model_family == "openvla":
        processor = get_processor(cfg)

    # Initialize local logging
    run_id = f"EVAL-{cfg.task_suite_name}-{cfg.model_family}-{DATE_TIME}"
    if cfg.run_id_note is not None:
        run_id += f"--{cfg.run_id_note}"
    os.makedirs(cfg.local_log_dir, exist_ok=True)
    local_log_filepath = os.path.join(cfg.local_log_dir, run_id + ".txt")
    log_file = open(local_log_filepath, "w")
    print(f"Logging to local log file: {local_log_filepath}")

    # Initialize Weights & Biases logging as well
    if cfg.use_wandb:
        wandb.init(
            entity=cfg.wandb_entity,
            project=cfg.wandb_project,
            name=run_id,
        )

    # Initialize LIBERO task suite
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[cfg.task_suite_name]()
    num_tasks_in_suite = task_suite.n_tasks
    print(f"Task suite: {cfg.task_suite_name}")
    log_file.write(f"Task suite: {cfg.task_suite_name}\n")

    # Get expected image dimensions
    resize_size = get_image_resize_size(cfg)

    return processor, log_file, task_suite, num_tasks_in_suite, resize_size

In [5]:
"""Main function to run the OpenVLA LIBERO inference demo."""
print("[*] Starting OpenVLA LIBERO Inference Demo")

# Initialize configuration
model_cfg = GenerateConfig()

# Setup model and configuration
print("[*] Loading model and setting up configuration...")
# model = setup_model_and_config(cfg)

model = get_vla_via_lora(model_cfg)

[*] Starting OpenVLA LIBERO Inference Demo
[*] Loading model and setting up configuration...
[*] Instantiating Pretrained VLA model
[*] Loading in BF16 with Flash-Attention Enabled


08/10 [22:52:11] WARNING  | >> Expected `transformers==4.40.1` and `tokenizers==0.19.1`   ]8;id=327442;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py\modeling_prismatic.py]8;;\:]8;id=525711;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla/openvla-7b/31f090d05236101ebfc381b61c674dd4746d4ce0/modeling_prismatic.py#228\228]8;;\
                          but got `transformers==4.53.3` and `tokenizers==0.21.4`; there                           
                          might be inference-time regressions due to dependency changes.                           
                          If in doubt, pleaseuse the above versions.                                               

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 18.39it/s]


In [6]:
# Setup logging and environment
print("[*] Setting up logging and environment...")
processor, log_file, task_suite, num_tasks_in_suite, resize_size = setup_logging_and_environment(model_cfg, model)

[*] Setting up logging and environment...
Logging to local log file: ./experiments/logs/EVAL-libero_10-openvla-2025_08_10-22_52_01.txt
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task suite: libero_10


In [7]:
ref_config = GenerateConfig(device = "cuda:3")
print(ref_config.device)
ref_model = get_model(ref_config)

cuda:3
[*] Instantiating Pretrained VLA model
[*] Loading in BF16 with Flash-Attention Enabled


08/10 [22:52:23] WARNING  | >> Expected `transformers==4.40.1` and `tokenizers==0.19.1`   ]8;id=94534;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25/modeling_prismatic.py\modeling_prismatic.py]8;;\:]8;id=554011;file:///home/zijianwang/.cache/huggingface/modules/transformers_modules/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25/modeling_prismatic.py#228\228]8;;\
                          but got `transformers==4.53.3` and `tokenizers==0.21.4`; there                           
                          might be inference-time regressions due to dependency changes.                           
                          If in doubt, pleaseuse the above versions.                                               

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 17.31it/s]


Loaded model: <class 'transformers_modules.openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25.modeling_prismatic.OpenVLAForActionPrediction'>


In [8]:
# Create dataset instance
dataset = TrajectoryDataset(model_cfg, model_cfg.winner_trajectory_path, model_cfg.task_suite_name, processor, device = model_cfg.device, model = model, stream_length = 3)

# dataset只返回"prompt_input_ids": [1, 2, 3], "chosen_input_ids": [4, 5], "rejected_input_ids": [6], 
# attention_mask在DataCollatorForPreference生成, labels在dpotrainer.concatenated_forward中生成. 

Found 212 success trajectories


In [28]:
from trl.trainer.dpo_trainer import DataCollatorForPreference
from torch.utils.data import DataLoader
model_cfg = GenerateConfig()
data_collator = DataCollatorForPreference(pad_token_id=processor.tokenizer.pad_token_id)
train_dataloader = DataLoader(
    dataset,
    batch_size=model_cfg.batch_size,
    shuffle=True,
    collate_fn=data_collator  # 需要返回上述格式的batch
)

In [ ]:
for batch in train_dataloader:
    print(batch['chosen_input_ids'])
    break

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tensor([[31884, 31944, 31917, 31895, 31877, 31904, 31744, 32001, 31920, 31914,
         31945, 31895, 31927, 31850, 31744, 32001, 31920, 31914, 31953, 31895,
         31931, 31855, 31744, 32001],
        [31833, 31844, 31812, 31914, 31877, 31916, 31872, 32001, 31833, 31844,
         31812, 31914, 31877, 31916, 31872, 32001, 31833, 31830, 31788, 31960,
         31900, 31914, 31872, 32001],
        [31958, 31878, 31938, 31895, 31877, 31924, 31872, 32001, 31958, 31878,
         31938, 31895, 31877, 31916, 31872, 32001, 31958, 31878, 31938, 31895,
         31877, 31921, 31872, 32001],
        [31887, 31878, 31873, 31895, 31877, 31896, 31744, 32001, 31884, 31903,
         31871, 31875, 31877, 31890, 31744, 32001, 31884, 31903, 31871, 31875,
         31877, 31890, 31744, 

In [16]:
print(batch.keys())
print(batch['chosen_input_ids'].shape)

dict_keys(['prompt_input_ids', 'prompt_attention_mask', 'chosen_input_ids', 'chosen_attention_mask', 'rejected_input_ids', 'rejected_attention_mask', 'pixel_values'])
torch.Size([4, 24])


In [17]:
import torch
with torch.no_grad():
    pred_test = model.forward(input_ids = batch["prompt_input_ids"], attention_mask = batch["chosen_attention_mask"], pixel_values=batch["pixel_values"])

print(pred_test.keys())

odict_keys(['logits', 'past_key_values', 'projector_features'])


In [38]:
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

def compute_log_probs(model, prompt_input_ids, completion_input_ids, prompt_attention_mask, completion_attention_mask, model_device=None):
    """计算模型对completion部分的log概率"""
    # 确保输入数据在正确的device上
    if model_device is None:
        model_device = next(model.parameters()).device
    
    prompt_input_ids = prompt_input_ids.to(model_device)
    completion_input_ids = completion_input_ids.to(model_device)
    prompt_attention_mask = prompt_attention_mask.to(model_device)
    completion_attention_mask = completion_attention_mask.to(model_device)
    
    # 拼接prompt和completion
    input_ids = torch.cat([prompt_input_ids, completion_input_ids], dim=1)
    attention_mask = torch.cat([prompt_attention_mask, completion_attention_mask], dim=1)
    
    # 前向传播
    with torch.no_grad() if model.training == False else torch.enable_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, use_cache=False)
        logits = outputs.logits
    
    # 计算completion部分的log概率
    prompt_len = prompt_input_ids.shape[1]
    completion_logits = logits[:, prompt_len-1:-1, :]  # 获取completion部分的logits
    completion_labels = completion_input_ids
    
    # 计算每个token的log概率
    log_probs = F.log_softmax(completion_logits, dim=-1)
    # 获取对应label的log概率
    selected_log_probs = torch.gather(log_probs, dim=-1, index=completion_labels.unsqueeze(-1)).squeeze(-1)
    # 对非padding的token求和
    sequence_log_probs = (selected_log_probs * completion_attention_mask).sum(dim=-1)
    
    return sequence_log_probs

def dpo_loss(policy_chosen_logps, policy_rejected_logps, 
             ref_chosen_logps, ref_rejected_logps, beta=0.1, target_device=None):
    """计算DPO损失，处理不同device上的tensor"""
    
    # 确定目标device (通常是policy model的device)
    if target_device is None:
        target_device = policy_chosen_logps.device
    
    # 将所有tensor移动到目标device
    policy_chosen_logps = policy_chosen_logps.to(target_device)
    policy_rejected_logps = policy_rejected_logps.to(target_device)
    ref_chosen_logps = ref_chosen_logps.to(target_device)
    ref_rejected_logps = ref_rejected_logps.to(target_device)
    
    # 计算log比率
    chosen_logratios = policy_chosen_logps - ref_chosen_logps
    rejected_logratios = policy_rejected_logps - ref_rejected_logps
    
    # DPO损失: -log(sigmoid(beta * (chosen_logratios - rejected_logratios)))
    logits = chosen_logratios - rejected_logratios
    losses = -F.logsigmoid(beta * logits)
    
    # 计算奖励（用于监控）
    chosen_rewards = beta * chosen_logratios.detach()
    rejected_rewards = beta * rejected_logratios.detach()
    
    return losses.mean(), chosen_rewards, rejected_rewards

def move_batch_to_device(batch, device):
    """将batch中的所有tensor移动到指定device"""
    moved_batch = {}
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            moved_batch[key] = value.to(device)
        else:
            moved_batch[key] = value
    return moved_batch

# 训练循环
def train_dpo(model, ref_model, train_dataloader, num_epochs=3, beta=0.1, lr=5e-6):
    """DPO训练主循环，支持不同device上的模型"""

    model_cfg = GenerateConfig()

        # Configure Unique Experiment ID & Log Directory
    exp_id = (
        f"{model_cfg.vla_path.split('/')[-1]}+{model_cfg.dataset_name}"
        f"+b{model_cfg.batch_size * model_cfg.grad_accumulation_steps}"
        f"+lr-{model_cfg.learning_rate}"
    )
    if model_cfg.use_lora:
        exp_id += f"+lora-r{model_cfg.lora_rank}+dropout-{model_cfg.lora_dropout}"
    import datetime
    exp_id += f"--{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
    if model_cfg.run_id_note is not None:
        exp_id += f"--{model_cfg.run_id_note}"

    run_dir, adapter_dir = os.path.join(model_cfg.run_root_dir, exp_id), os.path.join(model_cfg.adapter_tmp_dir, exp_id)
    wandb.init(entity=model_cfg.wandb_entity, project=model_cfg.wandb_project, name=f"dpo+{exp_id}")
    
    # 获取模型所在的device
    policy_device = next(model.parameters()).device
    ref_device = next(ref_model.parameters()).device
    
    print(f"Policy model device: {policy_device}")
    print(f"Reference model device: {ref_device}")
    
    # 设置优化器
    optimizer = AdamW(model.parameters(), lr=lr)
    
    # 确保参考模型不更新
    ref_model.eval()
    for param in ref_model.parameters():
        param.requires_grad = False
    
    # 训练循环
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(progress_bar):
            # 将batch移动到policy model的device
            policy_batch = move_batch_to_device(batch, policy_device)
            
            # 1. 计算策略模型的log概率 (在policy_device上)
            policy_chosen_logps = compute_log_probs(
                model, 
                policy_batch['prompt_input_ids'], 
                policy_batch['chosen_input_ids'], 
                policy_batch['prompt_attention_mask'], 
                policy_batch['chosen_attention_mask'],
                model_device=policy_device
            )
            policy_rejected_logps = compute_log_probs(
                model, 
                policy_batch['prompt_input_ids'], 
                policy_batch['rejected_input_ids'],
                policy_batch['prompt_attention_mask'], 
                policy_batch['rejected_attention_mask'],
                model_device=policy_device
            )
            
            # 2. 计算参考模型的log概率 (在ref_device上)
            with torch.no_grad():
                # 将batch移动到ref model的device
                ref_batch = move_batch_to_device(batch, ref_device)
                
                ref_chosen_logps = compute_log_probs(
                    ref_model, 
                    ref_batch['prompt_input_ids'], 
                    ref_batch['chosen_input_ids'],
                    ref_batch['prompt_attention_mask'], 
                    ref_batch['chosen_attention_mask'],
                    model_device = ref_device
                )
                ref_rejected_logps = compute_log_probs(
                    ref_model, 
                    ref_batch['prompt_input_ids'], 
                    ref_batch['rejected_input_ids'],
                    ref_batch['prompt_attention_mask'], 
                    ref_batch['rejected_attention_mask'],
                    model_device = ref_device
                )
            
            # 3. 计算DPO损失 (在policy_device上)
            loss, chosen_rewards, rejected_rewards = dpo_loss(
                policy_chosen_logps, policy_rejected_logps,
                ref_chosen_logps, ref_rejected_logps,
                beta=beta, target_device=policy_device
            )
            
            # 4. 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            # 梯度裁剪（可选）
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # 5. 计算指标
            accuracy = (chosen_rewards > rejected_rewards).float().mean()
            reward_margin = (chosen_rewards - rejected_rewards).mean()

            wandb.log({
                "loss": loss.item(),
                "accuracy": accuracy.item(),
                "reward_margin": reward_margin.item()
            })
            
            if batch_idx % 10 == 0:
                model.save_pretrained(adapter_dir)
                print(f"Saved adapter to {adapter_dir}, batch_idx: {batch_idx}, epoch: {epoch}")
                
            # 定期清理缓存（如果使用GPU）
            if batch_idx % 10 == 0:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
    


In [39]:
# 使用示例，处理不同device的情况
if __name__ == "__main__":
    # 示例：将ref_model放在CPU上以节省GPU内存
    
    # 或者在多GPU环境下，将模型放在不同GPU上
    # model = model.to('cuda:0')      # 策略模型在GPU 0
    # ref_model = ref_model.to('cuda:1')  # 参考模型在GPU 1
    
    # 开始训练
    train_dpo(model, ref_model, train_dataloader, num_epochs=1, beta=0.1, lr=model_cfg.learning_rate)

wandb: Currently logged in as: 15652388600 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


Policy model device: cuda:2
Reference model device: cuda:3


Epoch 1/1:   0%|          | 0/53 [00:00<?, ?it/s]

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:137: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_input_ids = [torch.tensor(example["prompt_input_ids"]) for example in examples]
/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  chosen_input_ids = [torch.tensor(example["chosen_input_ids"]) for example in examples]
/home/zijianwang/miniconda3/envs/openvla/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

...

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Epoch 1/1:  23%|██▎       | 12/53 [01:59<06:47,  9.93s/it]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f9cbc233e20>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f9cbc233940, execution_count=39 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7f9cbc233040, raw_cell="# 使用示例，处理不同device的情况
if __name__ == "__main__":
  .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bxuchang-lab1/hdd/zijianwang/openvla/vla-scripts/DPO/DPO_demo.ipynb#X20sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

# ---------------